# Camada Silver

In [0]:
CREATE SCHEMA IF NOT EXISTS workspace.projeto_vendas_silver;

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_silver.orders AS
SELECT
  order_id,
  customer_id,
  order_status,
  TRY_CAST(order_purchase_timestamp AS TIMESTAMP) AS purchase_ts,
  TRY_CAST(order_approved_at AS TIMESTAMP) AS approved_ts,
  TRY_CAST(order_delivered_customer_date AS TIMESTAMP) AS delivered_ts
FROM workspace.projeto_vendas_bronze.orders;

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_silver.customers AS
SELECT
  customer_id,
  customer_city,
  customer_state
FROM workspace.projeto_vendas_bronze.customers;


In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_silver.products AS
SELECT
  product_id,
  COALESCE(product_category_name, 'unknown') AS product_category
FROM workspace.projeto_vendas_bronze.products;


In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_silver.order_items AS
SELECT
  order_id,
  product_id,
  CAST(price AS DECIMAL(10,2)) AS price,
  CAST(freight_value AS DECIMAL(10,2)) AS freight_value
FROM workspace.projeto_vendas_bronze.order_items;


## Validação

In [0]:
SELECT * FROM workspace.projeto_vendas_silver.orders
LIMIT 10

In [0]:
SELECT * FROM workspace.projeto_vendas_silver.customers
LIMIT 10

In [0]:
SELECT * FROM workspace.projeto_vendas_silver.products
LIMIT 10

In [0]:
SELECT * FROM workspace.projeto_vendas_silver.order_items
LIMIT 10

## Verificando valores nulos

### Orders Table

Regra de negócio aplicada:
- Considerados apenas pedidos com status 'delivered' e data de entrega preenchida.
- Pedidos com data de entrega e data de aprovação nulas foram excluídos por inconsistência.


In [0]:
SELECT
  COUNT(*) AS total,
  COUNT(delivered_ts) AS entregues,
  COUNT(*) - COUNT(delivered_ts) AS nulos
FROM workspace.projeto_vendas_silver.orders;


In [0]:
SELECT
  order_id,
  order_status,
  purchase_ts,
  approved_ts,
  delivered_ts
FROM workspace.projeto_vendas_silver.orders
WHERE delivered_ts IS NULL;

In [0]:
SELECT
  COUNT(*) AS total,
  COUNT(approved_ts) AS aprovados,
  COUNT(*) - COUNT(approved_ts) AS nulos
FROM workspace.projeto_vendas_silver.orders;

In [0]:
SELECT
  order_id,
  order_status,
  purchase_ts,
  approved_ts,
  delivered_ts
FROM workspace.projeto_vendas_silver.orders
WHERE approved_ts IS NULL;

In [0]:
SELECT
  COUNT(*) AS total_linhas,
  COUNT(approved_ts) AS approved_nao_nulos,
  COUNT(delivered_ts) AS delivered_nao_nulos
FROM workspace.projeto_vendas_silver.orders
WHERE order_status = 'delivered';

Considerar apenas os pedidos válidos e entregues

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_silver.orders AS
SELECT
  order_id,
  customer_id,
  order_status,
  TRY_CAST(order_purchase_timestamp AS TIMESTAMP) AS purchase_ts,
  TRY_CAST(order_approved_at AS TIMESTAMP) AS approved_ts,
  TRY_CAST(order_delivered_customer_date AS TIMESTAMP) AS delivered_ts
FROM workspace.projeto_vendas_bronze.orders
WHERE order_status = 'delivered'
  AND order_delivered_customer_date IS NOT NULL;

In [0]:
SELECT
  COUNT(*) AS total_rows,
  COUNT(delivered_ts) AS non_null_values,
  COUNT(*) - COUNT(delivered_ts) AS null_values
FROM workspace.projeto_vendas_silver.orders;

In [0]:
DELETE FROM workspace.projeto_vendas_silver.orders
WHERE delivered_ts IS NULL;

In [0]:
SELECT
  COUNT(*) AS total_rows,
  COUNT(approved_ts) AS non_null_values,
  COUNT(*) - COUNT(approved_ts) AS null_values
FROM workspace.projeto_vendas_silver.orders;

In [0]:
DELETE FROM workspace.projeto_vendas_silver.orders
WHERE approved_ts IS NULL;

### Customers

In [0]:
SELECT
  COUNT(*) AS total_rows,
  COUNT(customer_id) AS customer_id_non_null,
  COUNT(*) - COUNT(customer_id) AS customer_id_null,
  COUNT(customer_city) AS customer_city_non_null,
  COUNT(*) - COUNT(customer_city) AS customer_city_null,
  COUNT(customer_state) AS customer_state_non_null,
  COUNT(*) - COUNT(customer_state) AS customer_state_null
FROM workspace.projeto_vendas_silver.customers;

### Order Items

In [0]:
SELECT
  COUNT(*) AS total_rows,
  COUNT(order_id) AS order_id_non_null,
  COUNT(*) - COUNT(order_id) AS order_id_null,
  COUNT(product_id) AS product_id_non_null,
  COUNT(*) - COUNT(product_id) AS product_id_null,
  COUNT(price) AS price_non_null,
  COUNT(*) - COUNT(price) AS price_null,
  COUNT(freight_value) AS freight_value_non_null,
  COUNT(*) - COUNT(freight_value) AS freight_value_null
FROM workspace.projeto_vendas_silver.order_items;

### Product Table

In [0]:
SELECT
  COUNT(*) AS total_rows,
  COUNT(product_id) AS order_id_non_null,
  COUNT(*) - COUNT(product_id) AS order_id_null,
  COUNT(product_category) AS product_id_non_null,
  COUNT(*) - COUNT(product_category) AS product_id_null
FROM workspace.projeto_vendas_silver.products


In [0]:
SELECT
  product_category,
  COUNT(*) AS itens_por_categoria
FROM workspace.projeto_vendas_silver.products
WHERE product_category = 'unknown'
GROUP BY product_category;